In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,Covid-19 News: Live Updates The New York Time...,https://www.nytimes.com/2020/09/09/world/covid...,"Wed, 09 Sep 2020 14:48:08 GMT",2020-09-09 14:48:08+00:00
8,Covid-19 drug rationed in the US is plentiful ...,https://www.cnn.com/2020/09/09/health/covid-re...,"Wed, 09 Sep 2020 13:17:00 GMT",2020-09-09 13:17:00+00:00
5,Escape The Pandemic: Summer Camps Offer COVID-...,https://www.npr.org/2020/09/09/910712563/escap...,"Wed, 09 Sep 2020 11:00:58 GMT",2020-09-09 11:00:58+00:00
3,"'So much for honor': Despite COVID cases, coll...",https://www.usatoday.com/in-depth/news/educati...,"Wed, 09 Sep 2020 06:17:00 GMT",2020-09-09 06:17:00+00:00
1,COVID-19 could reverse decades of progress tow...,https://www.who.int/news-room/detail/09-09-202...,"Wed, 09 Sep 2020 00:07:42 GMT",2020-09-09 00:07:42+00:00
4,Why COVID-19 is more deadly in people with obe...,https://www.sciencemag.org/news/2020/09/why-co...,"Tue, 08 Sep 2020 22:00:00 GMT",2020-09-08 22:00:00+00:00
6,“When Will We Have a Vaccine?” — Understanding...,https://www.nejm.org/doi/full/10.1056/NEJMp202...,"Tue, 08 Sep 2020 16:03:27 GMT",2020-09-08 16:03:27+00:00
9,Facial Masking for Covid-19 — Potential for “V...,https://www.nejm.org/doi/full/10.1056/NEJMp202...,"Tue, 08 Sep 2020 16:02:25 GMT",2020-09-08 16:02:25+00:00
7,COVID-19 Daily Update 9-8-2020 West Virginia ...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Tue, 08 Sep 2020 14:06:33 GMT",2020-09-08 14:06:33+00:00
2,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Tue, 08 Sep 2020 04:00:00 GMT",2020-09-08 04:00:00+00:00
